In [1]:
ls yelp_data/

bigram_model_all*                 sent_text_clean.csv*
bigram_sentences_all_health.txt*  sent_text.csv*
health_business00.csv*            trigram_bow_corpus_all.mm*
health_business01.csv*            trigram_bow_corpus_all.mm.index*
health_raw00.csv*                 trigram_dict_all.dict*
health_text.csv*                  trigram_model_all*
health_text_full.csv*             trigram_sentences_all_health.txt*
health_text_model_results.csv*    trigram_transformed_reviews_all_health.txt*
health_text_sentiment.csv*        unigram_sentences_all_health.txt*
health_text_sentiment_full.csv*   unigram_sentences_all.txt*
review_text_all - Copy.txt*       yelp_academic_dataset_business.json*
review_text_all.txt*              yelp_academic_dataset_review.json*
review_text_hospital.txt*


In [2]:
import os
import codecs

intermediate_directory = os.path.join( 'yelp_data')

review_txt_filepath = os.path.join(intermediate_directory,
                                   'review_text_hospital.txt')

In [3]:
%%time
import spacy
import pandas as pd
import itertools as it

nlp = spacy.load('en')

CPU times: user 4.12 s, sys: 1.72 s, total: 5.84 s
Wall time: 2.81 s


In [4]:
with codecs.open(review_txt_filepath, encoding='utf_8') as f:
    sample_review = list(it.islice(f, 8, 9))[0]
    sample_review = sample_review.replace('\\n', '\n')
        
print (sample_review)

My son is here recovering after major surgery. There have been complications and the nurses and doctors have been amazing!! They are allowing us to stay with him all day and have answered all of our questions honestly and patiently. Dr. Stephen Lewis his surgeon has gone out of his way to make my son's recovery as easy as possible. He visited my son on his day off! I was so thankful for that. It's still early in the process but we are extremely happy with the care we have been shown here. Wonderful hospital!



In [5]:
data = pd.read_csv(review_txt_filepath,sep='\n',names=['review'])

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12926 entries, 0 to 12925
Data columns (total 1 columns):
review    12926 non-null object
dtypes: object(1)
memory usage: 101.1+ KB


## Phrase Modeling

In [7]:
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

In [8]:
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space

def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    with codecs.open(filename, encoding='utf_8') as f:
        for review in f:
            yield review.replace('\\n', '\n')
            
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_review in nlp.pipe(line_review(filename),
                                  batch_size=10000, n_threads=4):
        
        for sent in parsed_review.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])

In [9]:
unigram_sentences_filepath = os.path.join(intermediate_directory,
                                          'unigram_sentences_all_health.txt')

In [10]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 1:

    with codecs.open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
        for sentence in lemmatized_sentence_corpus(review_txt_filepath):
            f.write(sentence + '\n')

CPU times: user 58min 44s, sys: 12min 36s, total: 1h 11min 21s
Wall time: 14min 11s


In [11]:
unigram_sentences = LineSentence(unigram_sentences_filepath)

In [12]:
for unigram_sentence in it.islice(unigram_sentences, 230, 240):
    print( ' '.join(unigram_sentence))

be only four -PRON- know that in time -PRON- would become immobile- so sad
so the choice be to replace -PRON- joint with a prosthetic steel joint
everyone who work there seem to love animal
everything be totally explain from begin to end
after surgery -PRON- learn that through the night katie be distraught and anxious
a tech actually get into katie 's cage and soothe with -PRON- until an anti anxiety med take effect
the only negative the procedure be very expensive prob be everywhere and -PRON- be difficult to actually meet with the doctor after surgery
nurse relay all necessary info but doctor spend the day in surgeries(the place be pack with pet with surgical need
-PRON- would not hesitate to recommend pvsec
-PRON- have to take -PRON- daughter to the er -PRON- just pull over at 3:57pm


In [13]:
bigram_model_filepath = os.path.join(intermediate_directory, 'bigram_model_all')

In [14]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute modeling yourself.
if 1 == 1:

    bigram_model = Phrases(unigram_sentences)

    bigram_model.save(bigram_model_filepath)
    
# load the finished model from disk
bigram_model = Phrases.load(bigram_model_filepath)

CPU times: user 7.19 s, sys: 719 ms, total: 7.91 s
Wall time: 7.93 s


In [15]:
bigram_sentences_filepath = os.path.join(intermediate_directory,
                                         'bigram_sentences_all_health.txt')

In [16]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 1:

    with codecs.open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
        for unigram_sentence in unigram_sentences:
            
            bigram_sentence = u' '.join(bigram_model[unigram_sentence])
            
            f.write(bigram_sentence + '\n')

/home/salas/miniconda3/envs/nlp/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 16.5 s, sys: 781 ms, total: 17.2 s
Wall time: 17.4 s


In [17]:
bigram_sentences = LineSentence(bigram_sentences_filepath)

In [18]:
for bigram_sentence in it.islice(bigram_sentences, 230, 240):
    print( ' '.join(bigram_sentence))

be only four -PRON- know that in time -PRON- would become immobile- so sad
so the choice be to replace -PRON- joint with a prosthetic steel joint
everyone who work there seem to love animal
everything be totally explain from begin to end
after surgery -PRON- learn that through the night katie be distraught and anxious
a tech actually get into katie 's cage and soothe with -PRON- until an anti_anxiety med take effect
the only negative the procedure be very expensive prob be everywhere and -PRON- be difficult to actually meet with the doctor after surgery
nurse relay all necessary info but doctor spend the day in surgeries(the place be pack with pet with surgical need
-PRON- would not hesitate to recommend pvsec
-PRON- have to take -PRON- daughter to the er -PRON- just pull over at 3:57pm


In [19]:
trigram_model_filepath = os.path.join(intermediate_directory,
                                      'trigram_model_all')

In [20]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute modeling yourself.
if 1 == 1:

    trigram_model = Phrases(bigram_sentences)

    trigram_model.save(trigram_model_filepath)
    
# load the finished model from disk
trigram_model = Phrases.load(trigram_model_filepath)

CPU times: user 7 s, sys: 828 ms, total: 7.83 s
Wall time: 7.81 s


In [21]:
trigram_sentences_filepath = os.path.join(intermediate_directory,
                                          'trigram_sentences_all_health.txt')

In [22]:
trigram_sentences_filepath

'yelp_data/trigram_sentences_all_health.txt'

In [23]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 1:

    with codecs.open(trigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
        for bigram_sentence in bigram_sentences:
            
            trigram_sentence = u' '.join(trigram_model[bigram_sentence])
            
            f.write(trigram_sentence + '\n')

/home/salas/miniconda3/envs/nlp/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 15.6 s, sys: 828 ms, total: 16.5 s
Wall time: 16.6 s


In [24]:
trigram_sentences = LineSentence(trigram_sentences_filepath)

In [25]:
for trigram_sentence in it.islice(trigram_sentences, 230, 240):
    print( ' '.join(trigram_sentence))

be only four -PRON- know that in time -PRON- would become immobile- so sad
so the choice be to replace -PRON- joint with a prosthetic steel joint
everyone who work there seem to love animal
everything be totally explain from begin to end
after surgery -PRON- learn that through the night katie be distraught and anxious
a tech actually get into katie 's cage and soothe with -PRON- until an anti_anxiety med take effect
the only_negative the procedure be very expensive prob be everywhere and -PRON- be difficult to actually meet with the doctor after surgery
nurse relay all necessary info but doctor spend the day in surgeries(the place be pack with pet with surgical need
-PRON- would not_hesitate to recommend pvsec
-PRON- have to take -PRON- daughter to the er -PRON- just pull over at 3:57pm


In [26]:
trigram_reviews_filepath = os.path.join(intermediate_directory,
                                        'trigram_transformed_reviews_all_health.txt')

In [27]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 1:

    with codecs.open(trigram_reviews_filepath, 'w', encoding='utf_8') as f:
        
        for parsed_review in nlp.pipe(line_review(review_txt_filepath),
                                      batch_size=10000, n_threads=4):
            
            # lemmatize the text, removing punctuation and whitespace
            unigram_review = [token.lemma_ for token in parsed_review
                              if not punct_space(token)]
            
            # apply the first-order and second-order phrase models
            bigram_review = bigram_model[unigram_review]
            trigram_review = trigram_model[bigram_review]
            
            # remove any remaining stopwords
            trigram_review = [term for term in trigram_review
                              if term not in spacy.en.STOPWORDS]
            
            # write the transformed review as a line in the new file
            trigram_review = u' '.join(trigram_review)
            f.write(trigram_review + '\n')

/home/salas/miniconda3/envs/nlp/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


AttributeError: module 'spacy' has no attribute 'en'

In [28]:
print ('Original:' + u'\n')

for review in it.islice(line_review(review_txt_filepath), 11, 12):
    print( review)

print ('----' + u'\n')
print ('Transformed:' + u'\n')

with codecs.open(trigram_reviews_filepath, encoding='utf_8') as f:
    for review in it.islice(f, 11, 12):
        print( review)

Original:

I had this misfortune of going to the ER here twice in the last year.  Once for me and once for the wife. Both were really BAD experiences in more ways than one.

I'll address my last visit which occurred a few weeks ago.   Without getting into the medical details, let me say the waiting room was completely empty when we arrived, so we got right in.  The layout if the patient rooms is L-shaped while nurses station in the center.   After a doctor did the initial examination, we had to wait maybe 45 minutes until more tests were done.  That is not a problem, but what was a problem is the loud nurses outside our room discussing everything from their dates last night, to other patients.  Yes, they were talking about patients, and NOT in a good way.  There was do door to close, just a curtain, so it was pretty hard NOT to hear it. Very unprofessional.

As we waited another doctor we had seen before entered our room and told us "good news" the test result was negative for this one

# Topic Modeling with Latent Dirichlet Allocation (LDA)

In [29]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis
import pyLDAvis.gensim
import warnings
import pickle as pickle

The first step to creating an LDA model is to learn the full vocabulary of the corpus to be modeled. We'll use gensim's Dictionary class for this.

In [30]:
trigram_dictionary_filepath = os.path.join(intermediate_directory,
                                           'trigram_dict_all.dict')

In [31]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to learn the dictionary yourself.
if 1 == 1:

    trigram_reviews = LineSentence(trigram_reviews_filepath)

    # learn the dictionary by iterating over all of the reviews
    trigram_dictionary = Dictionary(trigram_reviews)
    
    # filter tokens that are very rare or too common from
    # the dictionary (filter_extremes) and reassign integer ids (compactify)
    trigram_dictionary.filter_extremes(no_below=10, no_above=0.4)
    trigram_dictionary.compactify()

    trigram_dictionary.save(trigram_dictionary_filepath)
    
# load the finished dictionary from disk
trigram_dictionary = Dictionary.load(trigram_dictionary_filepath)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 3.29 ms


Like many NLP techniques, LDA uses a simplifying assumption known as the bag-of-words model. In the bag-of-words model, a document is represented by the counts of distinct terms that occur within it. Additional information, such as word order, is discarded.

Using the gensim Dictionary we learned to generate a bag-of-words representation for each review. The trigram_bow_generator function implements this. We'll save the resulting bag-of-words reviews as a matrix.

In the following code, "bag-of-words" is abbreviated as bow.

In [32]:
trigram_bow_filepath = os.path.join(intermediate_directory,
                                    'trigram_bow_corpus_all.mm')

In [33]:
def trigram_bow_generator(filepath):
    """
    generator function to read reviews from a file
    and yield a bag-of-words representation
    """
    
    for review in LineSentence(filepath):
        yield trigram_dictionary.doc2bow(review)

In [34]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to build the bag-of-words corpus yourself.
if 1 == 1:

    # generate bag-of-words representations for
    # all reviews and save them as a matrix
    MmCorpus.serialize(trigram_bow_filepath,
                       trigram_bow_generator(trigram_reviews_filepath))
    
# load the finished bag-of-words corpus from disk
trigram_bow_corpus = MmCorpus(trigram_bow_filepath)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 3.47 ms


In [35]:
trigram_bow_corpus


With the bag-of-words corpus, we're finally ready to learn our topic model from the reviews. We simply need to pass the bag-of-words matrix and Dictionary from our previous steps to LdaMulticore as inputs, along with the number of topics the model should learn. For this demo, we're asking for 50 topics.

In [36]:
lda_model_filepath = os.path.join(intermediate_directory, 'lda_model_all')

In [37]:
lda_model_filepath

'yelp_data/lda_model_all'

In [39]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to train the LDA model yourself.
if 1 == 1:

    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        
        # workers => sets the parallelism, and should be
        # set to your number of physical cores minus one
        lda = LdaMulticore(trigram_bow_corpus,
                           num_topics=50,
                           id2word=trigram_dictionary,
                           workers=3)
    
    lda.save(lda_model_filepath)
    
# load the finished LDA model from disk
lda = LdaMulticore.load(lda_model_filepath)

ValueError: cannot compute LDA over an empty collection (no terms)

In [2]:
def explore_topic(topic_number, topn=25):
    """
    accept a user-supplied topic number and
    print out a formatted list of the top terms
    """
        
    print (u'{:20} {}'.format(u'term', u'frequency') + u'\n')

    for term, frequency in lda.show_topic(topic_number, topn=25):
        print (u'{:20} {:.3f}'.format(term, round(frequency, 3)))

In [3]:
explore_topic(topic_number=0)

term                 frequency



NameError: name 'lda' is not defined